#**PYTHON** **CAPSTONE** **PROJECT**

#SMART HOME AUTOMATION

https://colab.research.google.com/drive/1QHLLhcZEFRR0RxuD2mLFT3GICgcQ9PxE?usp=sharing

In [1]:
#IMPORTING ALL REQUIRED LIBRARIES
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import csv
from datetime import datetime

In [2]:
#BASE CLASS DEVICE AND SUBCLASSES
class Device:
    def __init__(self, device_id, device_type, status):
        self.device_id = device_id
        self.device_type = device_type
        self.status = status

    def get_status(self):
        return self.status

    def set_status(self, status):
        self.status = status

    def to_dict(self):
        return {
            "device_id": self.device_id,
            "device_type": self.device_type,
            "status": self.status
        }

#SUBCLASSES
class Thermostat(Device):
    def __init__(self, device_id, status, temperature):
        super().__init__(device_id, 'Thermostat', status)
        self.temperature = temperature

    def set_temperature(self, temperature):
        self.temperature = temperature

    def get_temperature(self):
        return self.temperature

    def to_dict(self):
        data = super().to_dict()
        data["temperature"] = self.temperature
        return data

class Light(Device):
    def __init__(self, device_id, status, brightness):
        super().__init__(device_id, 'Light', status)
        self.brightness = brightness

    def set_brightness(self, brightness):
        self.brightness = brightness

    def get_brightness(self):
        return self.brightness

    def to_dict(self):
        data = super().to_dict()
        data["brightness"] = self.brightness
        return data

class SecurityCamera(Device):
    def __init__(self, device_id, status, recording):
        super().__init__(device_id, 'SecurityCamera', status)
        self.recording = recording

    def start_recording(self):
        self.recording = True

    def stop_recording(self):
        self.recording = False

    def to_dict(self):
        data = super().to_dict()
        data["recording"] = self.recording
        return data


In [ ]:
#MOUNTING THE DRIVE
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
#FUNCTION TO GENERATE RANDOM DATA
def generate_random_data():
    np.random.seed()
    device_data =[
    {
            'device_id':np.random.randint(1,1000) ,
            'device_type': np.random.choice(['Thermostat', 'Light', 'Security Camera']),
            'status': np.random.choice(['on', 'off']),
            'temperature': np.random.uniform(15.0, 25.0) if np.random.choice(['Thermostat', 'Light', 'Security Camera']) == 'Thermostat' else None,
            'brightness': np.random.randint(0, 100) if np.random.choice(['Thermostat', 'Light', 'Security Camera']) == 'Light' else None,
            'recording_status': np.random.choice(['recording', 'not recording']) if np.random.choice(['Thermostat', 'Light', 'Security Camera']) == 'Security Camera' else None,
            'timestamp': datetime.now().isoformat()
        }
    ]


    return device_data


#SAVING DEVICE DATA TO FILE
def log_device_data(filename, device_data):
    fieldnames = ['timestamp', 'device_id', 'device_type', 'status', 'temperature', 'brightness', 'recording_status']
    file_exists = os.path.isfile(filename)

    with open(filename, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if not file_exists or os.path.getsize(filename) == 0:
            writer.writeheader()
        for data in device_data:
            writer.writerow(data)

filename = "/content/drive/MyDrive/PYTHONCAPSTONE/Capstonepython_Sheet1.csv"
try:

  while True:
      device_data = generate_random_data()
      log_device_data(filename, device_data)
      time.sleep(10)
      print("The device data is exported to csv file")
except KeyboardInterrupt:
  print("Data logging stopped.")



The device data is exported to csv file
Data logging stopped.


In [4]:
#LOADING DEVICE DATA FROM THE FILE
def load_devices_from_file(filename):
    devices = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            print(row)
            if len(row) == 6:
                device_id, device_type, status, temperature, brightness, recording = row
                if device_type == 'Thermostat':
                    devices.append(Thermostat(device_id, status, float(temperature)))
                elif device_type == 'Light':
                    devices.append(Light(device_id, status, int(brightness)))
                elif device_type == 'SecurityCamera':
                    devices.append(SecurityCamera(device_id, status, recording == 'True'))
            else:
                print(f"Unexpected row length: {len(row)}. Row content: {row}")
    return devices

#TO CHECK THE CONTENT OF THE FILE
filename = "/content/drive/MyDrive/PYTHONCAPSTONE/Capstonepython_Sheet1.csv"
loaded_devices = load_devices_from_file(filename)
print(loaded_devices)

['2024-06-04T11:16:47.121833', '9', 'Thermostat', 'off', '16.791772017538438', '', '']
Unexpected row length: 7. Row content: ['2024-06-04T11:16:47.121833', '9', 'Thermostat', 'off', '16.791772017538438', '', '']
['2024-06-04T11:16:57.141390', '924', 'Security Camera', 'on', '22.5659998090175', '', 'not recording']
Unexpected row length: 7. Row content: ['2024-06-04T11:16:57.141390', '924', 'Security Camera', 'on', '22.5659998090175', '', 'not recording']
['2024-06-04T11:17:07.164953', '165', 'Light', 'off', '17.45513761433051', '', 'not recording']
Unexpected row length: 7. Row content: ['2024-06-04T11:17:07.164953', '165', 'Light', 'off', '17.45513761433051', '', 'not recording']
['2024-06-04T11:17:17.181795', '748', 'Light', 'off', '20.589442894034164', '8', 'not recording']
Unexpected row length: 7. Row content: ['2024-06-04T11:17:17.181795', '748', 'Light', 'off', '20.589442894034164', '8', 'not recording']
['2024-06-04T11:17:27.198628', '610', 'Light', 'off', '', '', 'not recordi

In [5]:
#DEVICE CONTROL LOGIC
def control_devices(devices, current_time):
    for device in devices:
        if(device is Light):
            if time(18, 0) <= current_time.time() <= time(6, 0):
                device.set_status('on')
            else:
                device.set_status('off')
        elif(device is Thermostat):
            if device.get_temperature() < 20:
                device.set_status('on')
            else:
                device.set_status('off')

In [6]:
#FUNCTION TO ANALYZE THE DATA
filename = "/content/drive/MyDrive/PYTHONCAPSTONE/Capstonepython_Sheet1.csv"
df = pd.read_csv(filename)
def analyze_data(df):
    if df.empty:
        print("DataFrame is empty. No data to analyze.")
        return None, None

    avg_temp = df[df['device_type'] == 'Thermostat']['temperature'].mean()
    peak_usage = df['timestamp'].value_counts().idxmax()
    return avg_temp, peak_usage


avg_temp, peak_usage = analyze_data(df)
print(f'Average Temperature for Thermostats: {avg_temp}')
print(f'Peak Usage Timestamp: {peak_usage}')

Average Temperature for Thermostats: 20.345570091946616
Peak Usage Timestamp: 2024-06-04T11:16:47.121833


In [7]:
#TO REMOVE THE NULL VALUES
df['temperature'].fillna(df['temperature'].mean(), inplace=True)
df['brightness'].fillna(df['brightness'].mean(), inplace=True)

In [8]:
#FUNCTION TO CREATE VISUALIZATION
def create_visualizations(df):
    plt.figure(figsize=(20, 20))

    #LINE CHART FOR TEMPERATURE CHANGES OVER TIME
    plt.subplot(4, 2, 1)
    thermostat_data = df[df['device_type'] == 'Thermostat']
    thermostat_data['temperature'].plot(kind='line', marker='o')
    plt.title('Temperature Over Time')
    plt.xlabel('Time')
    plt.ylabel('Temperature')
    plt.grid(True)

    #BAR CHART FOR ENERGY USAGE OF EACH DEVICE
    plt.subplot(4, 2, 2)
    df['device_type'].value_counts().plot(kind='bar',color=['green', 'blue', 'orange'])
    plt.title('Energy Usage by Device')
    plt.xlabel('Device Type')
    plt.ylabel('Count')

    #PIE CHART FOR THE PROPORTION OF TIME EACH DEVICE IS ACTIVE
    active_times = df[df['status'] == 'on'].groupby('device_type').size()
    total_times = df.groupby('device_type').size()
    active_proportions = (active_times / total_times).fillna(0)

     # Pie chart for recording status proportion (for Security Cameras)
    plt.subplot(4, 2, 3)
    security_data = df[df['device_type'] == 'Security Camera']
    recording_proportions = security_data['recording_status'].value_counts(normalize=True)
    recording_proportions.plot(kind='pie', autopct='%1.1f%%', colors=['yellow', 'purple'])
    plt.title('Recording Status Proportion (Security Cameras)')
# Bar chart for temperature distribution (for Thermostats)
    plt.subplot(4, 2, 4)
    thermostat_data['temperature'].plot(kind='hist', bins=10, color='blue', alpha=0.7)
    plt.title('Temperature Distribution')
    plt.xlabel('Temperature')
    plt.ylabel('Frequency')
    plt.grid(True)

 # Line chart for brightness changes over time (for Lights)
    plt.subplot(4, 2, 5)
    light_data = df[df['device_type'] == 'Light']
    light_data['brightness'].plot(kind='line', marker='o')
    plt.title('Brightness Over Time')
    plt.xlabel('Time')
    plt.ylabel('Brightness')
    plt.grid(True)

    plt.subplot(4, 2, 6)
    active_proportions.plot(kind='pie', autopct='%1.1f%%', colors=['red', 'green', 'orange'])
    plt.title('Active Time Proportion by Device Type')

    plt.tight_layout()
    plt.show()

    plt.tight_layout()
    plt.show()

#CREATE VISUALISATION
create_visualizations(df)
try:
    while True:
      time.sleep(10)
except KeyboardInterrupt:
  print("Data logging stopped.")


plt.show()



<Figure size 640x480 with 0 Axes>

Data logging stopped.
